## LSTM测试

In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from datetime import date
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import numpy as np
import requests
from requests.auth import HTTPBasicAuth
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from datetime import datetime, timedelta
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

import random
import os
import tensorflow as tf

/Users/loki/PycharmProjects/PythonProject5/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
today = date.today()

In [3]:
data = yf.download("CNYGBP=X", start="2015-01-01", end='2025-05-20')
all_test_df = data
all_test_df['ER'] = (all_test_df['High'] + all_test_df['Low']) / 2
all_test_df['inverse'] = 1/all_test_df['ER']
er_df = all_test_df[['inverse']].rename(columns={"Close": "CNY/GBP"})

# test average price


[*********************100%***********************]  1 of 1 completed

YF.download() has changed argument auto_adjust default to True


In [4]:
er_df = er_df.reset_index().rename(columns={'index': 'Date','inverse': 'ER'})
er_df['Date'] = pd.to_datetime(er_df['Date'])

In [5]:
df_weekday_aligned = er_df.copy()
df_weekday_aligned.tail(10)

Price,Date,ER
Ticker,,
2693,2025-05-06,9.694760
2694,2025-05-07,9.640645
2695,2025-05-08,9.614229
2696,2025-05-09,9.592418
2697,2025-05-12,9.568874
2698,2025-05-13,9.529573
2699,2025-05-14,9.601536
2700,2025-05-15,9.577489
2701,2025-05-16,9.580976


In [6]:
df_weekday_aligned['trade_war'] = 0
df_weekday_aligned.loc[(df_weekday_aligned['Date'] >= '2018-03-22') & (df_weekday_aligned['Date'] <= '2018-12-01'), 'trade_war'] = 1
df_weekday_aligned.loc[(df_weekday_aligned['Date'] >= '2025-02-01') , 'trade_war'] = 1

In [7]:
df_all = df_weekday_aligned

In [8]:
df_all['prev_change'] = df_all['ER'].pct_change().fillna(0)

# Create trend direction label (1 if next day up, 0 if down)
df_all['trend_up'] = (df_all['ER'].diff().shift(-1) > 0).astype(int)
df_all.dropna(inplace=True)  # drop last row if trend_up for next day is NaN

# Display the final feature set
display(df_all.tail(5))

Price,Date,ER,trade_war,prev_change,trend_up
Ticker,,,,,
2698,2025-05-13,9.529573,1,-0.004107,1
2699,2025-05-14,9.601536,1,0.007552,0
2700,2025-05-15,9.577489,1,-0.002504,1
2701,2025-05-16,9.580976,1,0.000364,1
2702,2025-05-19,9.622694,1,0.004354,0


In [9]:
lstm_df = df_all.drop(['prev_change'], axis=1)

lstm_df['Date'] = pd.to_datetime(lstm_df['Date'])  # 保证为 datetime 类型
lstm_df = lstm_df.set_index('Date')  # 设置为 index

In [10]:
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)

# 固定 TensorFlow 随机种子
tf.random.set_seed(42)

# 若用GPU，确保 TensorFlow 使用确定性算法
# 注意：这会牺牲一定速度
from tensorflow.keras import backend as K
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [11]:
lstm_df

Price,ER,trade_war,trend_up
Ticker,,,
Date,,,
2015-01-01,9.621861,0,0
2015-01-02,9.563429,0,0
2015-01-05,9.499834,0,0
2015-01-06,9.454924,0,0
2015-01-07,9.389671,0,0
...,...,...,...
2025-05-13,9.529573,1,1
2025-05-14,9.601536,1,0


In [13]:
# Split data into training and testing sets using 80/20 ratio
split_idx = int(len(lstm_df) * 0.91)
train_df = lstm_df.iloc[:split_idx]
test_df = lstm_df.iloc[split_idx:]

# Separate features (X) and target (y)
X_train = train_df # 注意：这里只取特征列
y_train = train_df['ER']
X_test = test_df
y_test = test_df['ER']

# Normalize features and target using Min-Max scaling (fit on training data only)
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()
X_train_scaled = feature_scaler.fit_transform(X_train)
X_test_scaled = feature_scaler.transform(X_test)
y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = target_scaler.transform(y_test.values.reshape(-1, 1))

# Prepare data for LSTM: create sequences of length `lookback` for each training example
lookback = 12  # use past 12 months to predict the next month
X_train_seq, y_train_seq = [], []
for i in range(lookback, len(X_train_scaled)):
    # sequence of `lookback` feature vectors
    X_train_seq.append(X_train_scaled[i-lookback:i])
    # target is the exchange rate at this time (one month ahead of last input month)
    y_train_seq.append(y_train_scaled[i])
X_train_seq = np.array(X_train_seq)
y_train_seq = np.array(y_train_seq)

# Prepare sequences for the test set (using preceding data for initial sequence)
X_test_seq, y_test_seq = [], []
# Concatenate train and test for sequence generation to include trailing train data in initial test sequences
total_X = np.concatenate((X_train_scaled, X_test_scaled), axis=0)
total_y = np.concatenate((y_train_scaled, y_test_scaled), axis=0)
train_len = len(X_train_scaled)
for i in range(train_len, len(total_X)):
    if i < lookback:
        continue  # skip until we have enough history
    X_test_seq.append(total_X[i-lookback:i])
    y_test_seq.append(total_y[i])
X_test_seq = np.array(X_test_seq)
y_test_seq = np.array(y_test_seq)

# Build the LSTM model
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
# 改进后的双层 LSTM 架构
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(lookback, X_train_seq.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dense(1))

# 别忘了这一步！！
model.compile(optimizer='adam', loss='mean_squared_error')

# 然后再训练
history = model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=16,validation_split=0.1, verbose=1 )

Epoch 1/50


/Users/loki/PycharmProjects/PythonProject5/.venv/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-05-25 12:21:53.175670: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{nod

129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0388 - val_loss: 4.7557e-04
Epoch 2/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0014 - val_loss: 4.5989e-04
Epoch 3/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0013 - val_loss: 3.1873e-04
Epoch 4/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0013 - val_loss: 2.8428e-04
Epoch 5/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0012 - val_loss: 3.0119e-04
Epoch 6/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0012 - val_loss: 2.6550e-04
Epoch 7/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0011 - val_loss: 2.5300e-04
Epoch 8/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0010 - val_loss: 2.5545e-04
Epoch 9/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0010 - val_loss: 2.7463e-04
Epoch 10/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.5156e-04 - val_loss: 3.4457e-04
Epoch 11/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.3755e-04 - val_loss: 2.6046e-04
Epoch 12/50

In [ ]:
X_test = test_df
y_test = test_df['ER']

# Normalize features and target using Min-Max scaling (fit on training data only)
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()
X_train_scaled = feature_scaler.fit_transform(X_train)
X_test_scaled = feature_scaler.transform(X_test)

In [16]:
X_pred_input = X_test_scaled[-lookback:]  # shape: (lookback, n_features)
X_pred_input = X_pred_input.reshape(1, lookback, X_pred_input.shape[1])  # reshape 成 LSTM 输入格式

# 预测归一化后的汇率值
y_pred_scaled = model.predict(X_pred_input)

# 反归一化成真实汇率
y_pred = target_scaler.inverse_transform(y_pred_scaled)

print(f"预测明日汇率：{float(y_pred[0][0]):.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
预测明日汇率：9.5533


2025-05-25 12:23:11.721976: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-05-25 12:23:11.722244: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),